In [4]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#read in data
root = "/Users/k23030440/S.-Pombe-MLPs - most recent/"
snps = pd.read_csv(root + 'Bence folder/QTL mapping/permutations/results_new_filtering.csv')
#washing phenotype /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/washing_phenotypes.csv
wash_pheno = pd.read_csv(root + 'Bence folder/QTL mapping/washing_phenotypes.csv')
#import rna seq corrs /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Analysis of Clement-Ziza RNA-seq/wash_corr.csv
wash_corr = pd.read_csv(root + 'Bence folder/Analysis of Clement-Ziza RNA-seq/wash_corr.csv')
#full snp dataset /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/SupplementaryDataset_S7_genotype.tsv
full_snps = pd.read_csv(root + 'internal data/updated_genotype_matrix_final_pos.tsv', sep='\t', header=0)
#load orthologs database /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Orthologs/final_orthology_database.csv
orthologs = pd.read_csv(root + 'Bence folder/Orthologs/final_orthology_database.csv')
full_snps_og = full_snps.copy()

In [5]:
#remove first 4 columns
full_snps = full_snps.iloc[:,4:]
#rename column 968 to X968
full_snps.rename(columns={'968':'X968'}, inplace=True)
#re order wash_pheno such that order of strains matches columns in full_snps column names
pca_pheno = pd.DataFrame(columns = ['strain','ratio'])
for i in full_snps.columns:
    #see what is the column name, and find the row in wash_pheno where strain has the same name
    
    row = wash_pheno.loc[wash_pheno['strain'] == i]
    pca_pheno = pd.concat([pca_pheno, row])
#pcapheno remove column Unnamed: 0
pca_pheno.drop(columns=['Unnamed: 0'], inplace=True)
#reset index
pca_pheno.reset_index(inplace=True)
#remove chromosome_ from chromosome column
snps['chromosome'] = snps['chromosome'].str.replace('chromosome_', '')
#change chromosome number to roman numbers
snps['chromosome'] = snps['chromosome'].str.replace('1','I')
snps['chromosome'] = snps['chromosome'].str.replace('2','II')
snps['chromosome'] = snps['chromosome'].str.replace('3','III')


/var/folders/g6/4flmdffs7l35hmmt1wkgrfcm0000gr/T/ipykernel_25014/3198399901.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pca_pheno = pd.concat([pca_pheno, row])


In [6]:
#show snps at chromosome_1 in position +- 10000 around 100011 in full_snps_og
full_snps_og.loc[(full_snps_og['chromosome'] == "chromosome_1") & (full_snps_og['position'] > 159850) & (full_snps_og['position'] < 179850)]



,chromosome,position,968_allele,Y0036_allele,968,Y0036,R1_1,R1_2,R1_3,R1_5,...,R1_9,R1_34,R1_44,R1_49,R1_50,R1_51,R1_52,R1_53,R1_54,R1_4
52,chromosome_1,162963,G,A,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
53,chromosome_1,163061,C,T,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
54,chromosome_1,164921,G,A,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
55,chromosome_1,165673,A,G,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
56,chromosome_1,168534,T,G,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
57,chromosome_1,170928,C,CT,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
58,chromosome_1,173079,A,C,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0
59,chromosome_1,173651,T,G,0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0


In [7]:
snps_t = snps.transpose()
#remove first 5 rows
snps_t = snps_t.iloc[5:]
#by matching index of snps_t to column strain in wash_pheno, we can add the phenotype to the snps_t dataframe
snps_t['phenotype'] = wash_pheno.set_index('strain')['ratio']
#rename all but last column to snps df position 
#snps_t all but last columns rename as snps["position"]
ncols = len(snps_t.columns)
snps_t.columns = np.append(snps["position"].values, snps_t.columns[ncols-1]).flatten()

In [8]:
#pandas read in "/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Pombe genome annotation.gff3"
import pandas as pd
df = pd.read_csv(root + 'external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.gff3', sep='\t', header=1)

#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]

In [10]:
#find a subset of df, where snps[position] is +- 1000 of df[start] or df[end]. Create a result df, and say which snps were there
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

result = pd.DataFrame()
for i in range(len(snps)):
    for j in range(len(df)):
        if snps['position'][i] >= df['start'][j] - 500 and snps['position'][i] <= df['end'][j] + 500:
            #and if chromsome is chromosome_2 for df chromsome II etc
            if snps['chromosome'][i] == df['chromosome'][j]:
                new_row = pd.DataFrame({'position': [snps['position'][i]], 
                        'ID': [df['ID'][j]], 
                        'Parent': [df['Parent'][j]], 
                        'object': [df['object'][j]], 
                        'chromosome': [df['chromosome'][j]], 
                        'start': [df['start'][j]], 
                        'end': [df['end'][j]], 
                        'strand': [df['strand'][j]], 
                        'detail': [df['detail'][j]]})

                result = pd.concat([result, new_row], ignore_index=True)
    

#if the last two characters are .1, remove them
for i in range(len(result)):
    if result['ID'][i][-2:] == ".1":
        result['ID'][i] = result['ID'][i][:-2]



In [11]:
#find the p_value of the snp position matches snps_t column_name
#add a column to result called p_value
pvals = np.array([])
for i in range(len(result)):
    for j in range(len(snps_t.columns)):
        #skip if colum nname is phenotype
        if snps_t.columns[j] == 'phenotype':
            continue
        if result['position'][i] == int(snps_t.columns[j]):
            pvals = np.append(pvals, snps_t.iloc[46,j])
result['p_value'] = pvals


In [12]:
#match ID with systematic name in wash_corr and add columns gene	correlation	p-value
#create np.arrays with NAs
corrs = np.array([np.nan]*len(result))
pvals = np.array([np.nan]*len(result))
genes = np.array([np.nan]*len(result), dtype=object)
for i in range(len(result)):
    for j in range(len(wash_corr)):
        if result['ID'][i] == wash_corr['systematic_names'][j]:
            corrs[i] = wash_corr['correlation'][j]
            pvals[i] = wash_corr['p-value'][j]
            genes[i] = wash_corr['gene'][j]

            


result['correlation'] = corrs
result['p_value_corr'] = pvals
result['gene'] = genes

In [13]:


import sys, re

f=open(root + 'external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.fa','r')
lines=f.readlines()

hre=re.compile('>(\S+)')
lre=re.compile('^(\S+)$')

regions={}

for line in lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in regions.keys()):
                        regions[id] += outl.group(1)
                else:
                        regions[id]  =outl.group(1)

In [17]:
#result_final___ only keep rows where snp is between start and end
result_final__ = result.copy()
result_final___ = result_final__.copy()
result_final___ = result_final___[(result_final___['position'] >= result_final___['start']-10) & (result_final___['position'] <= result_final___['end']+100)]

gene_exons = df.copy()
#only keep rows where object is CDS
gene_exons = gene_exons[gene_exons['object'] == 'CDS']
#only keep rows where ID is in result_final__['ID']
gene_exons_final = pd.DataFrame()
for i in np.unique(result_final___['ID']):
    df_subset = gene_exons[gene_exons['ID'] == str(i + ".1")]
    gene_exons_final = pd.concat([gene_exons_final, df_subset], ignore_index=True)
#reset index
gene_exons_final.reset_index(inplace=True)
#make a column exon_number
gene_exons_final['exon_number'] = np.nan
#for those where ID is multiple times, number the exons by increasing start if strand is +, or decreasing start if strand is -
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    if df_subset['strand'].values[0]== '+':
        df_subset = df_subset.sort_values(by=['start'], ascending=True)
    if df_subset['strand'].values[0] == '-':
        df_subset = df_subset.sort_values(by=['end'], ascending=False)
    df_subset['exon_number'] = np.arange(1, len(df_subset) + 1)
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset


#REMOVE entries wehre ID contains SPNCRNA
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].str.contains('SPNCRNA')]





#make a column called real_start
gene_exons_final['old_start'] = gene_exons_final['start']
gene_exons_final['old_end'] = gene_exons_final['end']
#for rows where strand is + look at the load the bases +- 5 around the start from regions
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset = df_subset[df_subset['exon_number'] == 1]
    if df_subset['strand'].values[0]== '+':
        seq = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        #where does ATG occur in seq
        real_start = seq.find('ATG')-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'start'] = real_start



    else:
        seq_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq_rev.find('CAT')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'end'] = real_end


    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    highest_exon = df_subset['exon_number'].max()
    df_subset = df_subset[df_subset['exon_number'] == highest_exon]
    if df_subset['strand'].values[0]== '+':

        seq2 = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq2.find('TAA')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'end'] = real_end
    else:
        seq2_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        real_start = seq2_rev.find('TCA')+2-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'start'] = real_start


    
    



#for each row calculate the length of the exon
gene_exons_final['exon_length'] = abs(gene_exons_final['end'] - gene_exons_final['start'])+1

#remove rows where ID is NaN
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].isna()]
#for each row calculate the length of the CDS until that exon starts
gene_exons_final['CDS_length_so_far'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    #order by exon_number
    df_subset = df_subset.sort_values(by=['exon_number'])
    
    df_subset['CDS_length_so_far'] = np.append(np.array(0),df_subset['exon_length'].cumsum())[:-1]
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset
#add total cds length
gene_exons_final['total_CDS_length'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset['total_CDS_length'] = sum(df_subset['exon_length'])
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset

gene_exons_final.head(10)

,index,chromosome,object,start,end,strand,Parent,ID,detail,exon_number,old_start,old_end,exon_length,CDS_length_so_far,total_CDS_length
0,0,II,CDS,2461357,2461495,-,SPBC12C2.08.1,SPBC12C2.08.1,exon,1.0,2461357,2461495,139,0.0,2347.0
1,1,II,CDS,2459018,2461225,-,SPBC12C2.08.1,SPBC12C2.08.1,exon,2.0,2459019,2461225,2208,139.0,2347.0
2,2,II,CDS,2452052,2452181,+,SPBC12C2.10c.1,SPBC12C2.10c.1,exon,1.0,2452052,2452181,130,0.0,4566.0
3,3,II,CDS,2452319,2456754,+,SPBC12C2.10c.1,SPBC12C2.10c.1,exon,2.0,2452319,2456757,4436,130.0,4566.0
4,4,II,CDS,2450850,2450854,-,SPBC12C2.11.1,SPBC12C2.11.1,exon,1.0,2450850,2450854,5,0.0,2094.0
5,5,II,CDS,2450711,2450781,-,SPBC12C2.11.1,SPBC12C2.11.1,exon,2.0,2450711,2450781,71,5.0,2094.0
6,6,II,CDS,2448488,2450505,-,SPBC12C2.11.1,SPBC12C2.11.1,exon,3.0,2448491,2450505,2018,76.0,2094.0
7,7,II,CDS,2446698,2447606,+,SPBC12C2.12c.1,SPBC12C2.12c.1,exon,1.0,2446698,2447606,909,0.0,909.0
8,8,II,CDS,2444329,2445924,+,SPBC12C2.13c.1,SPBC12C2.13c.1,exon,1.0,2444329,2445924,1596,0.0,1596.0
9,9,II,CDS,2304182,2305726,+,SPBC12D12.01.1,SPBC12D12.01.1,exon,1.0,2304182,2305726,1545,0.0,1545.0


In [18]:
#add to each row the unique number of positions that ID had
#add a column called unique
#remove duplicates of ID and position
#keep only entries with "object" is CDS
result = result[result['object'] == 'CDS']
result_final = result.drop_duplicates(subset=['ID', 'position'])
#reset index
result_final = result_final.reset_index(drop=True)
unique = np.array([])
for i in range(len(result_final)):
    #get the ID
    ID = result_final['ID'][i]
    #get the number of times ID appears in result
    count = len(result_final[result_final['ID'] == ID])
    #add count to unique
    unique = np.append(unique, count)
result_final['unique'] = unique



In [20]:
#show SPBC290.07 in result
#gather positions for each ID, and put in a single entry as a list. Add that to the df. THen remove duplicated IDs
result_final_ = result_final.groupby('ID')['position'].apply(list).reset_index(name='positions')
result_final__ = pd.merge(result_final, result_final_, on='ID')

#remove duplicates of ID
result_final__ = result_final__.drop_duplicates(subset=['ID'])

#reset index
result_final__ = result_final__.reset_index(drop=True)

In [21]:
#iterate through result_final__. If ID is found in orthologs, then add the albicans_name(s)cerevisiae_name(s) to the df
from numpy import dtype

albicans_names = np.array([np.nan]*len(result_final__),dtype=object)
cerevisiae_names = np.array([np.nan]*len(result_final__),dtype=object)
for i in range(len(result_final__)):
    for j in range(len(orthologs)):
        if result_final__['ID'][i] in orthologs['pombe_name(s)'][j]:
            albicans_names[i] = orthologs['albicans_name(s)'][j]
            cerevisiae_names[i] = orthologs['cerevisiae_name(s)'][j]
#add to df
result_final__['albicans_name(s)'] = albicans_names
result_final__['cerevisiae_name(s)'] = cerevisiae_names

In [22]:
#sort by p_value
result_final__ = result_final__.sort_values(by=['p_value_corr'])
result_final__.head(50)

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,unique,positions,albicans_name(s),cerevisiae_name(s)
27,2367068,SPBC19G7.09,SPBC19G7.09.1,CDS,II,2367117,2368823,+,exon,0.0,-0.373650,0.011462,ulp1,1.0,[2367068],NaN,NaN
49,2435580,SPBC21D10.06c,SPBC21D10.06c.1,CDS,II,2435636,2438482,+,exon,0.0,0.367721,0.012952,map4,5.0,"[2435580, 2436559, 2436911, 2437248, 2438335]",[nan],[nan]
77,41504,SPNCRNA.1097,SPNCRNA.1097.1,CDS,III,39627,41424,-,exon,0.0,0.364295,0.013886,SPNCRNA.1097,1.0,[41504],NaN,NaN
79,47493,SPCC757.03c,SPCC757.03c.1,CDS,III,47741,48475,-,exon,0.0,0.357839,0.015803,hsp3101,1.0,[47493],['C3_02610C_A'],['YDR533C' 'YMR322C' 'YOR391C' 'YPL280W']
48,2427851,SPBC21D10.09c,SPBC21D10.09c.1,CDS,II,2426785,2431588,+,exon,0.0,-0.343215,0.020990,rkr1,1.0,[2427851],NaN,NaN
34,2380243,SPNCRNA.1508,SPNCRNA.1508.1,CDS,II,2380550,2382306,-,exon,0.0,0.298155,0.046667,iws1-antisense-1,1.0,[2380243],NaN,NaN
40,2418555,SPBC1921.06c,SPBC1921.06c.1,CDS,II,2417166,2418302,-,exon,0.0,-0.286957,0.055978,pvg3,1.0,[2418555],NaN,NaN
61,2446853,SPBC12C2.12c,SPBC12C2.12c.1,CDS,II,2446698,2447606,+,exon,0.0,-0.283794,0.058861,glo1,3.0,"[2446853, 2447183, 2447678]",['C1_00500C_A'],['YML004C']
21,2334988,SPNCRNA.1505,SPNCRNA.1505.1,CDS,II,2335158,2337882,+,exon,1.0,0.267852,0.075255,SPBC24C6.09c-antisense-1,1.0,[2334988],NaN,NaN
16,2327939,SPBC24C6.05,SPBC24C6.05.1,CDS,II,2327506,2327536,+,exon,1.0,0.256126,0.089459,sec28,3.0,"[2327939, 2328246, 2328339]",['C5_00080C_A'],['YIL076W']


In [23]:
result_final__ = result.copy().reset_index(drop=True, inplace=False)

In [24]:
#iterate through positions in result_final__  and find the corresponding alleles in snps. Then add the two values in X968_allele	Y0036_allele to the df in two new columns at the start 
#add two columns to result_final__
result_final__['X968_allele'] = np.nan
result_final__['Y0036_allele'] = np.nan
for i in range(len(result_final__)):
    for j in range(len(snps)):
        if result_final__['position'][i] == snps['position'][j]:
            result_final__['X968_allele'][i] = snps['X968_allele'][j]
            result_final__['Y0036_allele'][i] = snps['Y0036_allele'][j]

In [25]:
#iterate through positions in result_final__ and look up the chromosome and position in regions, and find the corresponding letter +/- 5 bp in regions. Add it to a new column
result_final__['reference_seq']=np.nan
for i in range(len(result_final__)):
    for j in range(len(regions)):
        if result_final__['chromosome'][i] == list(regions.keys())[j]:
            #add | | characters left and right of tletter at position-1
            
            result_final__['reference_seq'][i] = regions[list(regions.keys())[j]][result_final__['position'][i]-4:result_final__['position'][i]-1] + ' |' + regions[list(regions.keys())[j]][result_final__['position'][i]-1] + '| ' + regions[list(regions.keys())[j]][result_final__['position'][i]:result_final__['position'][i]+3] 

#if strand is -, reverse reference_seq
for i in range(len(result_final__)):
    if result_final__['strand'][i] == '-':
        result_final__['reference_seq'][i] = result_final__['reference_seq'][i][::-1]
        #also flip the alleles
        result_final__['X968_allele'][i] = result_final__['X968_allele'][i][::-1]
        result_final__['Y0036_allele'][i] = result_final__['Y0036_allele'][i][::-1]
#also create reference_seq_ without the | | characters
result_final__['reference_seq_'] = result_final__['reference_seq'].str.replace('|', '')
result_final__['reference_seq_'] = result_final__['reference_seq_'].str.replace(' ', '')




In [26]:
#create column, whole gene as codons
result_final__['wt_gene'] = np.nan
#iterate through result_final__ positions. If object is CDS, then look up the sequence from start to end in regions.
#add the sequence to the df
for i in range(len(result_final__)):
    
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
        #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '-':
                
                gene = gene[::-1]
            
                gene = ''.join(gene)
            gene = 'A'*int(exon_len_so_far) + gene
            #write the gene as triplets of letters like AAA, ATC, etc
            #if divisible by 3
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[i:i+3] for i in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else: 
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            
                
                


            result_final__['wt_gene'][i] = gene

result_final__['mutant allele'] = np.nan
for i in range(len(result_final__)):
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
         #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '+':
                #decide which allele to use
                #check 
                #check how long the y0036 allele is 
                x968_len = len(result_final__['X968_allele'][i])
                y0036_len = len(result_final__['Y0036_allele'][i])
                
                mutation = result_final__['Y0036_allele'][i]
               

                gene = gene[:pos-start] + mutation + gene[pos-start+x968_len:]
            elif result_final__['strand'][i] == '-':
                x968_len = len(result_final__['X968_allele'][i])
                y0036_len = len(result_final__['Y0036_allele'][i])
                gene = gene[::-1]
                gene = ''.join(gene)
                mutation = result_final__['Y0036_allele'][i]
                
                gene = gene[:end-pos] + mutation + gene[end-pos+x968_len:]
                
            gene = 'A'*int(exon_len_so_far) + gene
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[k:k+3] for k in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else:
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            result_final__["mutant allele"][i] = gene
                
           


SPNCRNA.5536
SPNCRNA.5547
SPNCRNA.5597
SPNCRNA.5600
SPNCRNA.1503
SPNCRNA.1503
SPNCRNA.1504
SPNCRNA.1503
SPNCRNA.1504
SPNCRNA.1503
SPNCRNA.1504
SPNCRNA.1503
SPNCRNA.1504
SPNCRNA.1503
SPNCRNA.1504
SPNCRNA.1503
SPBC24C6.09c
SPNCRNA.1505
SPNCRNA.5612
SPNCRNA.5615
SPNCRNA.5615
SPBC19G7.09
SPNCRNA.1507
SPNCRNA.5616
SPNCRNA.5617
SPBC19G7.16
SPBC19G7.19
SPBC19G7.19
SPBC19G7.19
SPNCRNA.1508
SPNCRNA.5627
SPNCRNA.5627
SPNCRNA.5629
SPNCRNA.1512
SPNCRNA.5647
SPBC1921.07c
SPNCRNA.5647
SPBC1921.07c
SPNCRNA.5647
SPBC21D10.10
SPNCRNA.5652
SPNCRNA.5653
SPNCRNA.5654
SPNCRNA.5657
SPNCRNA.1513
SPNCRNA.1513
SPNCRNA.5659
SPNCRNA.5660
SPNCRNA.5661
SPNCRNA.5659
SPNCRNA.5660
SPNCRNA.5661
SPNCRNA.5659
SPNCRNA.5660
SPNCRNA.5661
SPNCRNA.5662
SPNCRNA.5660
SPNCRNA.5661
SPNCRNA.5662
SPNCRNA.1514
SPNCRNA.5662
SPNCRNA.1514
SPNCRNA.5663
SPNCRNA.5663
SPNCRNA.5663
SPSNORNA.38
SPNCRNA.1515
SPNCRNA.5664
SPNCRNA.1515
SPNCRNA.5664
SPNCRNA.1515
SPNCRNA.5664
SPNCRNA.5665
SPNCRNA.5665
SPBC12C2.09c
SPNCRNA.5669
SPNCRNA.5670
SPNCR

In [27]:
#CreATE A DF WHERE WHOLE GENE IS NOT NAN
stop_codon_df = result_final__[result_final__['wt_gene'].notna()]
stop_codon_df["stop_codon_predicted"] = np.nan
#iterate through mutant allele, and see whether it contains
#reset index
stop_codon_df = stop_codon_df.reset_index(drop=True)

for i in range(len(stop_codon_df)):
    for j in range(len(stop_codon_df['mutant allele'][i])):
        if stop_codon_df["strand"][i] == '+':
            if stop_codon_df['mutant allele'][i][j] == 'TAA' or stop_codon_df['mutant allele'][i][j] == 'TAG' or stop_codon_df['mutant allele'][i][j] == 'TGA':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
        else:
            if stop_codon_df['mutant allele'][i][j] == 'ATT' or stop_codon_df['mutant allele'][i][j] == 'ATC' or stop_codon_df['mutant allele'][i][j] == 'ACT':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
#show which entries have a stop codon predicted
stop_codon_df[stop_codon_df['stop_codon_predicted'] == 'yes']


,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,X968_allele,Y0036_allele,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted
3,2310884,SPNCRNA.5593,SPNCRNA.5593.1,CDS,II,2310853,2311405,+,exon,1.0,NaN,NaN,NaN,C,T,CAG |C| ATC,CAGCATC,"[TTC, ACA, GCA, TCT, AGG, GAA, GGA, GGT, TTC, ...","[TTC, ACA, GTA, TCT, AGG, GAA, GGA, GGT, TTC, ...",yes
5,2311107,SPNCRNA.5593,SPNCRNA.5593.1,CDS,II,2310853,2311405,+,exon,1.0,NaN,NaN,NaN,G,C,AAC |G| GAT,AACGGAT,"[ATA, GAA, ACG, GAT, GCT, TTG, TCA, TTT, TCC, ...","[ATA, GAA, ACC, GAT, GCT, TTG, TCA, TTT, TCC, ...",yes
7,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,1.0,-0.044401,0.772115,srb11,TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes
10,2326615,SPNCRNA.5602,SPNCRNA.5602.1,CDS,II,2325881,2326882,-,exon,1.0,NaN,NaN,NaN,C,A,ACT |C| TAG,ACTCTAG,"[GCT, ACT, CTA, GCG, TTT, GTC, GCA, GAT, TCA, ...","[GCT, ACT, ATA, GCG, TTT, GTC, GCA, GAT, TCA, ...",yes
18,2387677,SPNCRNA.5628,SPNCRNA.5628.1,CDS,II,2387488,2388232,-,exon,0.0,NaN,NaN,NaN,A,G,AAA |A| TAG,AAAATAG,"[CGC, TCA, AAA, TAG, TAG, TCA, ATT, TCG, AGA, ...","[CGC, TCA, AAG, TAG, TAG, TCA, ATT, TCG, AGA, ...",yes
22,2436559,SPNCRNA.5658,SPNCRNA.5658.1,CDS,II,2436317,2436671,-,exon,0.0,NaN,NaN,NaN,T,C,CCA |T| GAC,CCATGAC,"[CGT, TCC, ATG, ACC, ATC, GCT, AAC, AAC, TTC, ...","[CGT, TCC, ACG, ACC, ATC, GCT, AAC, AAC, TTC, ...",yes
27,2442808,SPRRNA.40,SPRRNA.40.1,CDS,II,2442700,2442852,-,exon,0.0,NaN,NaN,NaN,G,A,GGT |G| TGG,GGTGTGG,"[ATG, CCG, GTG, TGG, ATC, CGC, TTT, TGT, GGT, ...","[ATG, CCG, GTA, TGG, ATC, CGC, TTT, TGT, GGT, ...",yes


In [28]:
#in stop_codon_df create a column "possible frameshift" and fill it with nan. Then loop through X96_allele and Y0036_allele and check if the length is different. If yes and the difference is not divisible by 3, fill the column with "yes". Then show the entries where the column is "yes"
frameshift_df = stop_codon_df.copy()
frameshift_df['possible frameshift'] = np.nan

for i in range(len(frameshift_df)):
    if len(frameshift_df['X968_allele'][i]) != len(frameshift_df['Y0036_allele'][i]):
        if (len(frameshift_df['X968_allele'][i]) - len(frameshift_df['Y0036_allele'][i]))%3 != 0:
            frameshift_df['possible frameshift'][i] = 'yes'

In [29]:
#import a codon table as a dictionary
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}
#iterate through frameshift_df and trnslate the lists of triplets found in wt_gene and mutant allele
frameshift_df['wt_gene_translated'] = np.nan
frameshift_df['mutant allele_translated'] = np.nan
for i in range(len(frameshift_df)):
    #if strand is -, then change a to t and so on
    if frameshift_df['strand'][i] == '-':
        temp_wt_allele = frameshift_df['wt_gene'][i]
        temp_mut_allele = frameshift_df['mutant allele'][i]
        for j in range(len(temp_wt_allele)):
            temp_wt_allele[j] = temp_wt_allele[j].replace('A','t')
            temp_wt_allele[j] = temp_wt_allele[j].replace('T','a')
            temp_wt_allele[j] = temp_wt_allele[j].replace('C','g')
            temp_wt_allele[j] = temp_wt_allele[j].replace('G','c')
            temp_wt_allele[j] = temp_wt_allele[j].upper()
        for j in range(len(temp_mut_allele)):
            temp_mut_allele[j] = temp_mut_allele[j].replace('A','t')
            temp_mut_allele[j] = temp_mut_allele[j].replace('T','a')
            temp_mut_allele[j] = temp_mut_allele[j].replace('C','g')
            temp_mut_allele[j] = temp_mut_allele[j].replace('G','c')
            temp_mut_allele[j] = temp_mut_allele[j].upper()
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in temp_wt_allele])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in temp_mut_allele])
    else:
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['wt_gene'][i]])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['mutant allele'][i]])

In [49]:
#count how many times wt_gene_translated is not equal to mutant allele_translated when ID doesnt contain SPNC
frameshift_df[(frameshift_df['ID'].str.contains('RNA') == False) & (frameshift_df['wt_gene_translated'] != frameshift_df['mutant allele_translated'])].shape[0]

13

In [40]:
#display all cols
pd.set_option('display.max_columns', None)
#number of unique position, from rows where ID doesnt contain RNA
frameshift_df[frameshift_df['ID'].str.contains('RNA') == False]['position'].nunique()

26

In [32]:
#show where gene doesnt start with SPNC and possible frameshift
#show all rows and stop_codon_predicted
pd.set_option('display.max_rows', None)
frameshift_df[(frameshift_df['ID'].str.contains('SPNC') == False) & (frameshift_df['stop_codon_predicted'] == 'yes')]

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,X968_allele,Y0036_allele,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift,wt_gene_translated,mutant allele_translated
7,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,1.0,-0.044401,0.772115,srb11,TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes,yes,YMLKKNEEKGFSLEALVATCIYLSCKV,YMLKKMRKRVFLWRR*WLHAFIYHAKW
27,2442808,SPRRNA.40,SPRRNA.40.1,CDS,II,2442700,2442852,-,exon,0.0,NaN,NaN,NaN,G,A,GGT |G| TGG,GGTGTGG,"[TAC, GGC, CAC, ACC, TAG, GCG, AAA, ACA, CCA, ...","[TAC, GGC, CAT, ACC, TAG, GCG, AAA, ACA, CCA, ...",yes,NaN,YGHT*AKTPVPVRSLQLSV*GPR*YYGW,YGHT*AKTPVPVRSLQLSV*GPR*YYGW


In [ ]:
%load_ext watermark
%watermark


Last updated: 2023-11-05T10:32:25.211706+00:00

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 7.31.1

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 21.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [ ]:
%watermark --iversions



matplotlib: 3.8.0
seaborn   : 0.13.0
pandas    : 1.4.4
sys       : 3.9.0 | packaged by conda-forge | (default, Nov 26 2020, 07:54:06) 
[Clang 11.0.0 ]
re        : 2.2.1
scipy     : 1.11.3
numpy     : 1.23.4

